In [7]:
using ITensors
using ITensorChemistry

In [11]:
function create_molecule(m::String, r::Float64, rep::Int64)
    s = []
    for j in 0:rep-1
        push!(s,Atom(m, j*r , 0.0, 0.0))
    end
    return Molecule(s)
end

c_create_molecule (generic function with 1 method)

In [ ]:
function energy_at_bond(r)
  # define molecule geometry
  molecule = create_molecule("H",r,4)
  # build electronic hamiltonian 
  hf = molecular_orbital_hamiltonian(molecule; basis="sto-3g")
  hamiltonian = hf.hamiltonian
  hartree_fock_state = hf.hartree_fock_state
  hartree_fock_energy = hf.hartree_fock_energy
  # hilbert space
  s = siteinds("Electron", 4; conserve_qns=true)
  H = MPO(hamiltonian, s)
  # initialize MPS to HF state
  ψhf = MPS(s, hartree_fock_state)
  # run dmrg
  dmrg_kwargs = (;
    nsweeps=10,
    maxdim=[10,20,30,40,50,100],
    cutoff=1e-8,
    noise=[1e-6, 1e-7, 1e-8, 0.0],
  )
  dmrg_energy, _ = dmrg(H, ψhf; nsweeps=10, outputlevel=0)
  return hartree_fock_energy, dmrg_energy
end


In [10]:
# bond distances
r⃗ = 0.7:0.1:1
energies = []
for r in r⃗
  push!(energies, (r,energy_at_bond(r)))
end

Molecule
Atom 1: H,   r⃗ = (0.0, 0.0, 0.0)
Atom 2: H,   r⃗ = (0.3, 0.0, 0.0)
Atom 3: H,   r⃗ = (0.6, 0.0, 0.0)
Atom 4: H,   r⃗ = (0.8999999999999999, 0.0, 0.0)
Atom 5: H,   r⃗ = (1.2, 0.0, 0.0)


In [ ]:
function min_energy(energy)
    dm_energy = []
    for j in energy
        push!(dm_energy, j[2][2])
    end
    min = minimum(dm_energy)
    for j in energy
        if j[2][2] == min
            return j
        end
    end
end

In [ ]:
# using ITensorsGPU
# using CUDA
# CUDA.versioninfo()
# CUDA.functional()
# gpu = cu